### Run this in Python kernel

To reset font style of output (in case it is not the default one)

In [1]:
%%html
<style>
body {
    font-family: "Arial", cursive, sans-serif; # font style of application
}
pre {
    font-family: "Courier New"; # font style of cell outputs
}
</style>  

### Run below in the backtesting_clojure kernel

In [1]:
; import libraries from kernel
(ns clojure-backtesting.demo
  (:require [clojure-backtesting.data :refer :all]
            [clojure-backtesting.data-management :refer :all]
            [clojure-backtesting.portfolio :refer :all]
            [clojure-backtesting.order :refer :all]
            [clojure-backtesting.evaluate :refer :all]
            [clojure-backtesting.plot :refer :all]
            [clojure-backtesting.counter :refer :all]
            [clojure-backtesting.automation :refer :all]
            [clojure-backtesting.parameters :refer :all]
            [clojure-backtesting.indicators :refer :all]
            [clojure-backtesting.direct :refer :all]
            [clojure.string :as str]
            [clojure.java.io :as io]
            [clojure.pprint :as pprint]
  ) ;; require all libriaries from core
)

nil

### Import dataset

In [2]:
; path to dataset = "/Volumes/T7/CRSP"
; change it to the relative path to your own dataset
;
(load-dataset "/Volumes/T7/CRSP" "main" add-aprc)

The dataset is already furnished by add-aprc. No more modification is needed.


"Date range: 1972-01-03 ~ 2017-02-10"

### Initialise portfolio

In [3]:
;; initialise with current date and initial capital (= $10000)
(init-portfolio "1980-12-16" 10000)

### Write a strategy

The following code implements a simple trading strategy:

In a timespan of 10 days (inclusive of today),
- Buy 50 stocks of AAPL on the first day
- Sell 10 stocks of AAPL on every other day

### (1) Running the strategy
- Note that you should have loaded the **extract** dataset

In [4]:
;; define the "time span", i.e. to trade in the coming 10 days 
(def num-of-days (atom 10))                              

(while (pos? @num-of-days) ;; check if num-of-days is > 0
    (do 
        ;; write your trading strategy here
        (if (= 10 @num-of-days) ;; check if num-of-days == 10
            (do
                (order "14593" 50 :print true) ; buy 50 stocks
                ;; (println ((fn [date] (str "Buy 50 stocks of AAPL on " date)) (get-date)))
            )
        )
        (if (odd? @num-of-days) ;; check if num-of-days is odd
            (do
                (order "14593" -10 :print true) ; sell 10 stocks
                ;; (println ((fn [date] (str "Sell 10 stocks of AAPL on " date)) (get-date)))
            )
        )
        
        (update-eval-report) ;; update the evaluation metrics every day
        (println (get-date))
        ; move on to the next trading day
        (next-date)
        
        ; decrement counter
        (swap! num-of-days dec)
    )
)

1980-12-16
Order: 1980-12-17 | 14593 | 50.000000.
1980-12-17
Order: 1980-12-18 | 14593 | -10.000000.
1980-12-18
1980-12-19
Order: 1980-12-22 | 14593 | -10.000000.
1980-12-22
1980-12-23
Order: 1980-12-24 | 14593 | -10.000000.
1980-12-24
1980-12-26
Order: 1980-12-29 | 14593 | -10.000000.
1980-12-29
1980-12-30
Order: 1980-12-31 | 14593 | -10.000000.


nil

### Check order record

In [6]:
(print-order-record)


|      :date |  :tic |  :price | :aprc | :quantity |
|------------+-------+---------+-------+-----------|
| 1980-12-17 | 14593 | 25.9375 | 25.35 |        50 |
| 1980-12-18 | 14593 | 26.6875 | 25.67 |       -10 |
| 1980-12-22 | 14593 | 29.6875 | 26.88 |       -10 |
| 1980-12-24 | 14593 | 32.5625 | 27.98 |       -10 |
| 1980-12-29 | 14593 | 36.0625 | 29.25 |       -10 |
| 1980-12-31 | 14593 | 34.1875 | 28.58 |       -10 |


nil

### Check portfolio record

In [8]:
;; view final portfolio
(print-portfolio)


| :asset | :price | :aprc | :quantity | :tot-val |
|--------+--------+-------+-----------+----------|
|   cash |    N/A |   N/A |       N/A | 10116.11 |


nil

In [9]:
;; view portfolio value and return
(print-portfolio-record)


|      :date | :tot-value | :daily-ret | :tot-ret | :loan | :leverage |
|------------+------------+------------+----------+-------+-----------|
| 1980-12-16 |  $10000.00 |      0.00% |    0.00% | $0.00 |      0.00 |
| 1980-12-17 |  $10000.00 |      0.00% |    0.00% | $0.00 |      0.00 |
| 1980-12-18 |  $10016.25 |      0.00% |    0.07% | $0.00 |      0.00 |
| 1980-12-19 |  $10043.73 |      0.12% |    0.19% | $0.00 |      0.00 |
| 1980-12-22 |  $10066.29 |      0.00% |    0.29% | $0.00 |      0.00 |
| 1980-12-23 |  $10081.29 |      0.06% |    0.35% | $0.00 |      0.00 |
| 1980-12-24 |  $10100.29 |      0.00% |    0.43% | $0.00 |      0.00 |
| 1980-12-26 |  $10122.77 |      0.10% |    0.53% | $0.00 |      0.00 |
| 1980-12-29 |  $10126.41 |      0.00% |    0.55% | $0.00 |      0.00 |
| 1980-12-30 |  $10123.21 |     -0.01% |    0.53% | $0.00 |      0.00 |
| 1980-12-31 |  $10119.51 |     -0.02% |    0.52% | $0.00 |      0.00 |


nil

### Generate evaluation report

In [10]:
(print-eval-report)


|      :date | :tot-value |    :vol |  :r-vol |  :sharpe | :r-sharpe | :pnl-pt | :max-drawdown |
|------------+------------+---------+---------+----------+-----------+---------+---------------|
| 1980-12-17 |     $10000 | 0.0000% | 0.0000% |  0.0000% |   0.0000% |      $0 |        0.0000 |
| 1980-12-18 |     $10015 | 0.0000% | 0.0000% |  0.0000% |   0.0000% |      $7 |        0.0000 |
| 1980-12-19 |     $10042 | 0.0578% | 0.0578% |  3.1854% |   3.1854% |     $21 |      100.0000 |
| 1980-12-22 |     $10064 | 0.0517% | 0.0517% |  5.3929% |   5.3929% |     $21 |      100.0000 |
| 1980-12-23 |     $10078 | 0.0490% | 0.0490% |  6.9722% |   6.9722% |     $26 |      100.0000 |
| 1980-12-24 |     $10097 | 0.0461% | 0.0461% |  9.1301% |   9.1301% |     $24 |      100.0000 |
| 1980-12-26 |     $10119 | 0.0491% | 0.0491% | 10.4957% |  10.4957% |     $29 |      100.0000 |
| 1980-12-29 |     $10122 | 0.0473% | 0.0473% | 11.2136% |  11.2136% |     $24 |      100.0000 |
| 1980-12-30 |     $10119 | 0

nil

### Plot variables
Below are example codes that show how to plot different variables in the portfolio record / evaluation record.

### 1. Portfolio daily return

In [11]:
(def data (deref portfolio-value))

#'clojure-backtesting.demo/data

In [12]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "port-value")
  data))

#'clojure-backtesting.demo/data-to-plot

In [13]:
(first data-to-plot)

{:date "1980-12-16", :tot-value 10000, :daily-ret 0.0, :tot-ret 0.0, :loan 0.0, :leverage 0.0, :margin 0.0, :plot "port-value"}

In [14]:
(plot data-to-plot :plot :date :daily-ret false)

### 2. Plot volatility

In [15]:
(def data (deref eval-record))

#'clojure-backtesting.demo/data

In [16]:
(first data)

{:date "1980-12-17", :tot-value 10000.0, :vol 0.0, :r-vol 0.0, :sharpe 0.0, :r-sharpe 0.0, :pnl-pt 0.0, :max-drawdown 0.0}

In [17]:
; Add legend name to series
(def data-to-plot
 (map #(assoc % :plot "volatility")
  data))

#'clojure-backtesting.demo/data-to-plot

In [20]:
(print-eval-report)


|      :date | :tot-value |    :vol |  :r-vol |  :sharpe | :r-sharpe | :pnl-pt | :max-drawdown |
|------------+------------+---------+---------+----------+-----------+---------+---------------|
| 1980-12-17 |     $10000 | 0.0000% | 0.0000% |  0.0000% |   0.0000% |      $0 |        0.0000 |
| 1980-12-18 |     $10015 | 0.0000% | 0.0000% |  0.0000% |   0.0000% |      $7 |        0.0000 |
| 1980-12-19 |     $10042 | 0.0578% | 0.0578% |  3.1854% |   3.1854% |     $21 |      100.0000 |
| 1980-12-22 |     $10064 | 0.0517% | 0.0517% |  5.3929% |   5.3929% |     $21 |      100.0000 |
| 1980-12-23 |     $10078 | 0.0490% | 0.0490% |  6.9722% |   6.9722% |     $26 |      100.0000 |
| 1980-12-24 |     $10097 | 0.0461% | 0.0461% |  9.1301% |   9.1301% |     $24 |      100.0000 |
| 1980-12-26 |     $10119 | 0.0491% | 0.0491% | 10.4957% |  10.4957% |     $29 |      100.0000 |
| 1980-12-29 |     $10122 | 0.0473% | 0.0473% | 11.2136% |  11.2136% |     $24 |      100.0000 |
| 1980-12-30 |     $10119 | 0

nil

In [19]:
(plot data-to-plot :plot :date :vol true)

### 3. Plot sharpe ratio

In [21]:
; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "sharpe ratio")
  data))

(plot data-to-plot :plot :date :sharpe true)

### 4. Plot stock price

In [21]:
(def data (deref order-record))

#'clojure-backtesting.demo/data

In [22]:
(first data)

{:date "1980-12-17", :tic "AAPL", :price 25.9375, :aprc "26.09", :quantity 50}

In [23]:
; Add rename legend
(def data-to-plot
 (map #(assoc % :plot "price")
  data))

(plot data-to-plot :plot :date :price true)